### Markowitz Mean -Variance Optimizer for strategic asset Allocation
---

Ce projet est un mini code qui implémente en python l'optimiseur de type mean-variance de Markowitz.
Il peut servir de point de départ pour plein de projet, dans le cadre de mon stage par exemple, je suis parti de ce code 
et j'ai ensuite explorer comment on pouvait intégrer des objectifs climatiques comme la réduction carbone ou l'analyse des scénarios climatiques dans l'allocation stratégoque d'actifs. 

Ici la seule contrainte est que les poids des actifs somment à 100 mais on peut bien adapté le code si on a besoin de limiter les poids de certains classes d'actifs comme les private Equity par exemple.

Pour les graphiques , j'ai utilisé plotly, qui contrairement à matplotlib permet d'avoir des graphiques interactifs et plus jolis
(avis personnel)


In [38]:
pip install --upgrade nbformat


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
pip install -U kaleido


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


##### Libraries set up
---

In [40]:
# Libraries

import pandas as pd 
import numpy as np 
import scipy.optimize as sc 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt


import datetime as dt 
import yfinance as yf 

#### Preliminary: Useful function: data importation- Hypothèses de rendement-hypothèses des risques - portfolio performance
---

In [41]:
# Data Importation

def getData(stocks,start,end):
    """
    stocks := represents a list of companies' symbols that we are interest to
    start and end := enable to define the period through which we want to recuperate
    stocks prices 

    """
    stock_data = yf.download(stocks, start, end)
    stock_data = stock_data['Close'] # We can retrieve several pieces of information(High, Low, Open, Volume), but we choose to keep the close price
    returns = stock_data.pct_change(fill_method=None)
 # Method for obtaining daily returns
    mean_return = returns.mean()
    cov_matrix = returns.cov()
    return mean_return, cov_matrix
    

In [42]:
# preliminary steps: Performance Method
def portfolioPerformance(weights, mean_return, cov_matrix):
    portfolio_return = np.sum(mean_return*weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))*np.sqrt(252)
    return portfolio_return, portfolio_std

#### Optimization process
---

In [43]:
# Step1: Maximum Sharpe Ratio
""" 1. Maximum Sharpe Ratio: Maximize the return with a given level of volatility.
    2. In order to use the function from the scipy package, we will reduce it to a minimization problem,
    so we need to implement the Negative Sharpe Ratio
    
"""
def negativeSR(weights, mean_return, cov_matrix, risk_free_rate = 0 ):
    p_return, p_std = portfolioPerformance(weights, mean_return, cov_matrix)
    return -( p_return-risk_free_rate)/p_std 


In [44]:
# Step2: Optimization

def maxSR(mean_return, cov_matrix, risk_free_rate=0, constraint_set=(0,1)):
    # We want to minimize the function negativeSR
    num_assets= len(mean_return)
    args= (mean_return, cov_matrix, risk_free_rate)
    # The constraints to our minimization pb is that the weights must sum up to 1
    constraints= ({'type': 'eq',
                   'fun': lambda x: np.sum(x)-1
                   })
    bound= constraint_set
    bounds= tuple(bound for asset in range(num_assets))
    # the function "minimize" minimizes the function up to his first attribute
    result= sc.minimize(negativeSR, num_assets*[1/ num_assets],
                       args=args, method= 'SLSQP', bounds= bounds,
                       constraints= constraints
                        )
    return result

In [45]:
# Minimize the portfolio Variance

#1 function to minimize
def portfolioVariance(weights, mean_return, cov_matrix):
    return portfolioPerformance(weights, mean_return, cov_matrix)[1]

In [46]:
#2 optimization
def minimizeVariance(mean_return, cov_matrix, constraint_set=(0,1)):
    num_assets= len(mean_return)
    args= (mean_return, cov_matrix)
    constraints= ({'type': 'eq',
                   'fun': lambda x: np.sum(x)-1
                   })
    bound= constraint_set
    bounds= tuple(bound for asset in range(num_assets))
    
    result= sc.minimize(portfolioVariance, num_assets*[1/ num_assets],
                       args=args, method= 'SLSQP', bounds= bounds,
                       constraints= constraints
                        )
    return result           

In [47]:
# Step3 Optimal Portfolio
""" For a given return, we are looking for a portfolio with a minimal variance"""
def portfolioReturn(weights, mean_return, cov_matrix):
    return portfolioPerformance(weights, mean_return, cov_matrix)[0]

In [48]:
#2
def efficientPortfolio(mean_return, cov_matrix, return_target, constraint_set=(0,1)):
    num_assets= len(mean_return)
    args= (mean_return, cov_matrix)
    constraints= ({'type': 'eq',
                   'fun': lambda x: portfolioReturn(x,mean_return,cov_matrix)-return_target},
                  {'type': 'eq',
                   'fun': lambda x: np.sum(x)-1
                   })
                  
    bound= constraint_set
    bounds= tuple(bound for asset in range(num_assets))
    
    result= sc.minimize(portfolioVariance, num_assets*[1/ num_assets],
                       args=args, method= 'SLSQP', bounds= bounds,
                       constraints= constraints
                        )
    return result           
    

#### Results to return-strategic asset allocation
----

In [49]:
# Step4: Efficient Portfolio
def calculateResults(mean_return, cov_matrix,risk_free_rate=0, constraint_set=(0,1)):
    
    # Max SR Portfolio
    maxSR_portfolio= maxSR(mean_return, cov_matrix)
    maxSR_return, maxSR_std= portfolioPerformance(maxSR_portfolio['x'], mean_return, cov_matrix)
    
    maxSR_allocation= pd.DataFrame(maxSR_portfolio['x'], index= mean_return.index, columns= ['Allocation'])
    maxSR_allocation.Allocation=[round(i*100,0) for i in maxSR_allocation.Allocation]
    
    # Min volatility Portfolio
    min_vol_portfolio= minimizeVariance(mean_return,cov_matrix)
    min_vol_return, min_vol_std= portfolioPerformance(min_vol_portfolio['x'], mean_return, cov_matrix)
    
    min_vol_allocation= pd.DataFrame(min_vol_portfolio['x'], index= mean_return.index, columns= ['Allocation'])
    min_vol_allocation.Allocation= [round(i*100,0) for i in min_vol_allocation.Allocation]
    
    # Efficient Frontiere
    efficient_list=[]
    target_return= np.linspace(min_vol_return, maxSR_return, 20)
    for target in target_return:
        efficient_list.append(efficientPortfolio(mean_return, cov_matrix, target)['fun'])
    
    min_vol_return, min_vol_std= round(min_vol_return*100,2), round(min_vol_std*100,2)
    maxSR_return,maxSR_std= round(maxSR_return*100,2), round(maxSR_std*100,2)
    
    return maxSR_return, maxSR_std, maxSR_allocation, min_vol_return, min_vol_std, min_vol_allocation, efficient_list, target_return
    
    

In [54]:
stock_list = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'NVDA', 'NFLX', 'DIS', 'V', 'JNJ', 'PG', 'JPM', 'BAC', 'XOM']
start_Date = dt.date(2020, 3, 1)
end_Date = dt.date(2021, 3, 1)
mean_return, cov_matrix = getData(stock_list, start_Date, end_Date)
maxSR_allocation = calculateResults(mean_return, cov_matrix)[2]

# Afficher les allocations pour le portefeuille Max SR
print("Max Sharpe Ratio Portfolio Allocation:\n", maxSR_allocation)


[*********************100%%**********************]  15 of 15 completed

1 Failed download:
['FB']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Max Sharpe Ratio Portfolio Allocation:
         Allocation
Ticker            
AAPL           7.0
AMZN           7.0
BAC            7.0
DIS            7.0
FB             7.0
GOOGL          7.0
JNJ            7.0
JPM            7.0
MSFT           7.0
NFLX           7.0
NVDA           7.0
PG             7.0
TSLA           7.0
V              7.0
XOM            7.0


#### Graphics
----

In [51]:
def EF_graph(mean_return, cov_matrix, risk_free_rate=0, constraint_set=(0, 1)):
    maxSR_return, maxSR_std, maxSR_allocation, min_vol_return, min_vol_std, min_vol_allocation, efficient_list, target_return = calculateResults(mean_return, cov_matrix, risk_free_rate, constraint_set)

    # MaxSR
    max_sharpe_ratio = go.Scatter(
        name='Maximum Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_return],
        marker=dict(color='red', size=14, line=dict(width=3, color='black'))
    )

    # Min vol
    min_vol = go.Scatter(
        name='Minimum Volatility',
        mode='markers',
        x=[min_vol_std],
        y=[min_vol_return],
        marker=dict(color='blue', size=14, line=dict(width=3, color='black'))
    )

    # Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(eff_std * 100, 2) for eff_std in efficient_list],
        y=[round(target * 100, 2) for target in target_return],
        line=dict(color='green', width=4, dash='dashdot')
    )

    # layout
    data = [max_sharpe_ratio, min_vol, EF_curve]
    layout = go.Layout(
        title='Portfolio Optimization with the Efficient Frontier',
        yaxis=dict(title='Annualized Return (%)'),
        xaxis=dict(title='Annualized Volatility (%)'),
        showlegend=True,
        legend=dict(x=0.75, y=0, traceorder='normal',
                    bgcolor='#E2E2E2', bordercolor='black',
                    borderwidth=2)
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()